In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

from transformers import TFRobertaModel
from transformers import AutoTokenizer

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
filename = "/kaggle/input/uzbek-news-preprocessed-datasets/combined_uznews_30k_preprocessed_ready_for_modeling.csv"
df = pd.read_csv(filename)
df.head()

,text,sentiment
0,milliy elektron ommaviy axborot vositalari ass...,positive
1,wizzair abu dabi arzon aviakompaniyasi abu dab...,positive
2,yildan beri yuvinmagan dunyoning eng kir odami...,negative
3,o‘zbekiston birinchi kuzgi sovuqni kutmoqda du...,positive
4,iyundan tibbiyot xodimlarining oylik maoshi os...,positive


In [3]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})
df.head()

,text,sentiment
0,milliy elektron ommaviy axborot vositalari ass...,1
1,wizzair abu dabi arzon aviakompaniyasi abu dab...,1
2,yildan beri yuvinmagan dunyoning eng kir odami...,0
3,o‘zbekiston birinchi kuzgi sovuqni kutmoqda du...,1
4,iyundan tibbiyot xodimlarining oylik maoshi os...,1


## Class Balancing by RandomOverSampler

In [4]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text']).reshape(-1, 1), np.array(df['sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text', 'sentiment']);

In [5]:
train_os['sentiment'].value_counts()

1    23849
0    23849
Name: sentiment, dtype: int64

## Train - Validation - Test split

In [6]:
X = train_os['text'].values
y = train_os['sentiment'].values

In [7]:
seed=42

# split the original data into train and TEST sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

# split the remaining 90% of the data into train and VALIDATION sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=seed)

## One hot encoding

In [8]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()
y_test_le = y_test.copy()

In [9]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [10]:
print(f"TRAINING DATA: {X_train.shape[0]}\nVALIDATION DATA: {X_valid.shape[0]}\nTESTING DATA: {X_test.shape[0]}" )

TRAINING DATA: 38635
VALIDATION DATA: 4293
TESTING DATA: 4770


# UzRoberta

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('rifkat/uztext-3Gb-BPE-Roberta')

In [12]:
MAX_LEN=512

In [13]:
# tokenize the training data
train_inputs = tokenizer(X_train.tolist(), max_length=MAX_LEN, padding='max_length', truncation=True)
train_input_ids = np.array(train_inputs['input_ids'])
train_attention_masks = np.array(train_inputs['attention_mask'])

# tokenize the validation data
val_inputs = tokenizer(X_valid.tolist(), max_length=MAX_LEN, padding='max_length', truncation=True)
val_input_ids = np.array(val_inputs['input_ids'])
val_attention_masks = np.array(val_inputs['attention_mask'])

# tokenize the test data
test_inputs = tokenizer(X_test.tolist(), max_length=MAX_LEN, padding='max_length', truncation=True)
test_input_ids = np.array(test_inputs['input_ids'])
test_attention_masks = np.array(test_inputs['attention_mask'])

# Modeling

In [14]:
def create_model(roberta_model, max_len=MAX_LEN):
    
    opt = tf.keras.optimizers.Adam(
        learning_rate=1e-5, 
        beta_1=0.9, 
        beta_2=0.999, 
        epsilon=1e-7
    )
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = roberta_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    return model

In [15]:
roberta_model = TFRobertaModel.from_pretrained('rifkat/uztext-3Gb-BPE-Roberta', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

In [16]:
model = create_model(roberta_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  83449344   ['input_1[0][0]',                
 el)                            thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [17]:
model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=16)

Epoch 1/4
2415/2415 [==============================] - 1437s 586ms/step - loss: 0.4864 - categorical_accuracy: 0.7602 - val_loss: 0.3831 - val_categorical_accuracy: 0.8325
Epoch 2/4
2415/2415 [==============================] - 1404s 581ms/step - loss: 0.2975 - categorical_accuracy: 0.8759 - val_loss: 0.2885 - val_categorical_accuracy: 0.8880
Epoch 3/4
2415/2415 [==============================] - 1404s 581ms/step - loss: 0.1519 - categorical_accuracy: 0.9441 - val_loss: 0.2649 - val_categorical_accuracy: 0.9131
Epoch 4/4
2415/2415 [==============================] - 1405s 582ms/step - loss: 0.0769 - categorical_accuracy: 0.9721 - val_loss: 0.2845 - val_categorical_accuracy: 0.9269


# Results

In [18]:
result = model.predict([test_input_ids,test_attention_masks])

150/150 [==============================] - 53s 342ms/step


In [19]:
y_pred = np.zeros_like(result)
y_pred[np.arange(len(y_pred)), result.argmax(1)] = 1

In [20]:
print('\tClassification Report:\n\n',classification_report(y_test,y_pred, target_names=['Negative', 'Positive']))

	Classification Report:

               precision    recall  f1-score   support

    Negative       0.91      0.97      0.94      2431
    Positive       0.97      0.90      0.93      2339

   micro avg       0.94      0.94      0.94      4770
   macro avg       0.94      0.94      0.94      4770
weighted avg       0.94      0.94      0.94      4770
 samples avg       0.94      0.94      0.94      4770



## Test with user input

In [ ]:
# import re
#
# def clean_text(text):
#     if not isinstance(text, str):
#         return ''
#
#     # Remove URLs
#     text = re.sub(r'http\S+', '', text)
#     # Remove digits
#     text = re.sub(r'\d+', '', text)
#
#     # exception: replace / - _ with space
#     text = re.sub(r'/', ' ', text)
#     text = re.sub(r'[-_]', ' ', text)
#
#     # Replace all ' with ‘ except for cases where it appears as a single character by itself
#     text = re.sub(r"(?<!\w)'(?!\w)|'", '‘', text)
#
#     # Remove Russian letters
#     text = re.sub(r'[А-Яа-я]+', '', text)
#
#     # Remove all non-word characters except for ‘ and convert to lowercase
#     text = re.sub(r"[^\w\s‘’]+(?<!‘)", "", text).lower()
#
#     # Remove extra whitespace
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

In [28]:
# while True:
#     input_sentence = input("Enter a sentence (or 'exit' to quit): ")
#
#     if input_sentence.lower() == 'exit':
#         print("Exiting...")
#         break
#
#     clean_sentence = clean_text(input_sentence)
#
#     input_ids = tokenizer.encode(clean_sentence, add_special_tokens=True)
#
#     input_ids = input_ids[:MAX_LEN]
#
#     padding_length = MAX_LEN - len(input_ids)
#     input_ids = input_ids + ([0] * padding_length)
#
#     input_ids = np.array([input_ids])
#
#     attention_masks = np.where(input_ids != 0, 1, 0)
#
#     pred_probs = model.predict([input_ids, attention_masks])[0]
#
#     pred_class = np.argmax(pred_probs)
#
#     if pred_class == 0:
#         print("Negative: probability of negative class is", pred_probs[0])
#     else:
#         print("Positive: probability of positive class is", pred_probs[1])

Enter a sentence (or 'exit' to quit):  Toshkentning aholisi eng ko'p bo'lgan tumanlari ma'lum qilindi  Batafsil 👉 https://uzreport.news/society/toshkentning-aholisi-eng-kop-bolgan-tumanlari-malum-qilindi  @uzreport_uz


1/1 [==============================] - 0s 43ms/step
Positive: probability of positive class is 0.9463582


Enter a sentence (or 'exit' to quit):  Nemolchi.uz loyihasi muallifi o‘ziga o‘lim tahdidi bo‘lgani haqida xabar berdi  U shu tufayli O‘zbekistonni vaqtincha tark etganini ma’lum qilgan.  👉 https://kun.uz/uz/19304075  Kun.uz rasmiy kanali (https://t.me/kunuz)


1/1 [==============================] - 0s 42ms/step
Negative: probability of negative class is 0.9849635


Enter a sentence (or 'exit' to quit):  3 oyda iste'molchilarga 418 milliondan ortiq elektron chek berildi  Batafsil 👉 https://uzreport.news/society/3-oyda-istemolchilarga-418-milliondan-ortiq-elektron-chek-berildi  @uzreport_uz


...

1/1 [==============================] - 0s 42ms/step
Positive: probability of positive class is 0.99921036


Enter a sentence (or 'exit' to quit):  Dahshat: Buxoroda cho‘l hududida 13 yoshli qiz to‘rt kishi tomonidan zo‘rlandi.  Qiz akasining oilasi bilan yashagan. Akasining xotini uni kattalar bilan jinsiy aloqa qilishga majburlagan. U qizni to'rt kishiga berdi.  Erkaklar qizni cho‘l hududidagi tashlandiq binoga olib borib, uchovi navbatma-navbat zo‘rlagan.


1/1 [==============================] - 0s 42ms/step
Negative: probability of negative class is 0.9991033


Enter a sentence (or 'exit' to quit):  exit


Exiting...


In [ ]:
# # save the model
# model.save('uznews_sentiment_model_tf_92acc.h5')

In [29]:
# !mkdir -p saved_model
# model.save('saved_model/uznews_sentiment_model_tf')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
# import shutil
# shutil.make_archive('tf_uznews_sentiment_model', 'zip', '/kaggle/working/saved_model/uznews_sentiment_model_tf')

'/kaggle/working/tf_uznews_sentiment_model.zip'